# Analyses Fabian Sanchez
Studentennr: 202187874

Groep: 2 VC


### Prerequisites

In [161]:
pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\sfsan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [162]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error

from sqlalchemy import create_engine

In [163]:
# verbinding
server = r"DESKTOP-1DRSSG0\SQLEXPRESS" 
database = "DEP1_DWH"

engine = create_engine("mssql+pyodbc://@{}/{}?driver=ODBC+Driver+17+for+SQL+Server".format(server, database))

## Gemakkelijk 2

### Opdracht

We beschikken voor de dataset van 1300 residentiële gebruikers jammer genoeg niet over de
overeenkomstige piekverbruiken.
Daarom zullen we als benadering voor de maandpiek het volgende gebruiken, namelijk het maximum
verbruik voor één kwartier per maand * 4.
We willen verifiëren of dat wel een goede benadering is.

Maak gebruik van de 2 bijhorende bestanden (van gebruiker 1301) om te controleren of deze aanpak
wel een goede benadering is. Je moet de data van gebruiker 1301 niet toevoegen aan de DWH.
Bepaal zelf welke metriek je hierbij best kan gebruiken. Denk daarbij aan de metrieken die aan bod
komen in ML

### Preprocess piekvermogens

In [ ]:
historiek_piekvermogen_1301 = pd.read_csv("../data/input/Sprint2/Historiek_piekvermogen_elektriciteit_1301_20240201_20250331.csv", sep=";")
historiek_piekvermogen_1301.drop(columns=["Van (tijdstip)", "Tot (tijdstip)", "Meter", "Validatiestatus", "Omschrijving"], inplace=True)
historiek_piekvermogen_1301.head()


In [ ]:
historiek_piekvermogen_1301['Volume'] = historiek_piekvermogen_1301['Volume'].str.replace(",", ".")

historiek_piekvermogen_1301['Volume'] = historiek_piekvermogen_1301['Volume'].astype(float)

historiek_piekvermogen_1301["Van (datum)"] = pd.to_datetime(historiek_piekvermogen_1301["Van (datum)"], format='%d-%m-%Y')

historiek_piekvermogen_1301.head()

In [ ]:
historiek_piekvermogen_1301 = sqldf('''
                                    select Volume as MaandPiek,
                                        strftime('%Y-%m', "Van (datum)") as maand
                                    from historiek_piekvermogen_1301   
                                  ''')

historiek_piekvermogen_1301.head(10)

In [ ]:
historiek_piekvermogen_1301.head(25)

### Preprocess verbruikshistoriek

In [ ]:
verbruikshistoriek_1301 = pd.read_csv("../data/input/Sprint2/Verbruikshistoriek_elektriciteit_1301_20240513_20250303_kwartiertotalen.csv", sep=";")

verbruikshistoriek_1301.head()

In [ ]:
verbruikshistoriek_1301["Van (datum)"] = pd.to_datetime(verbruikshistoriek_1301["Van (datum)"], format='%d-%m-%Y')
verbruikshistoriek_1301.head(20)

In [ ]:
verbruikshistoriek_1301 = sqldf('''
                                    select max(Volume) as max_volume,
                                          strftime('%Y-%m', "Van (datum)") as maand 
                                   from verbruikshistoriek_1301
                                   where Register like 'Afname%'
                                   group by maand
                                   order by maand
                                '''    )

In [ ]:
verbruikshistoriek_1301.head(10)    

In [ ]:
verbruikshistoriek_1301['max_volume'] = verbruikshistoriek_1301['max_volume'].str.replace(",", ".")

verbruikshistoriek_1301['max_volume'] = verbruikshistoriek_1301['max_volume'].astype(float) * 4

In [ ]:
verbruikshistoriek_1301.drop(labels=[10], inplace=True)

In [ ]:

verbruikshistoriek_1301.head(15)

### Merge

In [ ]:

df_vergelijking = pd.merge(historiek_piekvermogen_1301, verbruikshistoriek_1301, on='maand')
df_vergelijking = df_vergelijking.rename(columns={'max_volume': 'MaxVolume', 'maand': 'Maand'})
df_vergelijking = df_vergelijking[['Maand', 'MaandPiek', 'MaxVolume']]
df_vergelijking.head(15)

### Visuele voorstelling

In [ ]:
x = np.arange(len(df_vergelijking))

plt.figure(figsize=(15, 7))
plt.bar(x - 0.2, df_vergelijking['MaandPiek'], width=0.4, label='MaandPiek')
plt.bar(x + 0.2, df_vergelijking['MaxVolume'], width=0.4, label='MaxVolume')

plt.xticks(x, df_vergelijking['Maand'], rotation=20)
plt.legend()
plt.show()


### Statistische metrieken

In [ ]:
waarnemingen = df_vergelijking['MaandPiek']
voorspellingen = df_vergelijking['MaxVolume']

rmse = root_mean_squared_error(waarnemingen, voorspellingen)
mape = mean_absolute_percentage_error(waarnemingen, voorspellingen)

print("RMSE:",rmse)
print("MAPE:",mape)

### Conclusie

Het is aan de hand van zowel de histogram als de RMSE en MAPE waarden duidelijk dat de voorspellingen van de piekvermogen zeer goed zijn.